In [1]:
import json, requests
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import BadRequestError, NotFoundError
import warnings
warnings.filterwarnings("ignore")
from string import Template

In [2]:
client = Elasticsearch (
    "https://127.0.0.1:9200",
    verify_certs= False,
    basic_auth=('elastic', 'elastic'),
    request_timeout=300)

In [3]:
index_list = client.cat.indices(format="json")
index_list

ListApiResponse([{'health': 'green', 'status': 'open', 'index': 'mastodon_social-000001', 'uuid': '5Phbn0C4T3eSn6lgvQExEg', 'pri': '3', 'rep': '1', 'docs.count': '10000', 'docs.deleted': '0', 'store.size': '8.7mb', 'pri.store.size': '4.3mb'}, {'health': 'green', 'status': 'open', 'index': 'mastodon_social-000002', 'uuid': 'dvAAvnd9QS21SiBqj3FDfA', 'pri': '3', 'rep': '1', 'docs.count': '4120', 'docs.deleted': '0', 'store.size': '4mb', 'pri.store.size': '2mb'}, {'health': 'green', 'status': 'open', 'index': 'epa_air-000001', 'uuid': 'ENoZGLX1R3-wHnLowgWGxA', 'pri': '3', 'rep': '1', 'docs.count': '10079', 'docs.deleted': '3', 'store.size': '2.1mb', 'pri.store.size': '1mb'}, {'health': 'green', 'status': 'open', 'index': 'epa_air-000002', 'uuid': 'MoriUUTcRmWe_mDMBKpXQQ', 'pri': '3', 'rep': '1', 'docs.count': '3870', 'docs.deleted': '0', 'store.size': '1.2mb', 'pri.store.size': '656.1kb'}, {'health': 'green', 'status': 'open', 'index': 'vic_roads_crash', 'uuid': 'uecBFYBZTM2oRVi8sAyrpA', '

In [5]:
query= {
    "query":{
        "bool":{
            "must":{
                "match":{
                    "_id":"IOQHNI8Bj0R9TQ6uLc3V"
                }
            }
        }

    }
}
res = client.search(index="bom_weather", body=query)

In [6]:
res

ObjectApiResponse({'took': 1099, 'timed_out': False, '_shards': {'total': 12, 'successful': 12, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'bom_weather-000001', '_id': 'IOQHNI8Bj0R9TQ6uLc3V', '_score': 1.0, '_source': {'STATION': 'Melbourne (Olympic Park)', 'STATE': 'Victoria', 'DATE': '2024-04-30T23:30:00', 'Location': {'lat': -37.8, 'lon': 145.0}, 'wmo': 95936, 'air_temp': 11.5, 'apparent_t': 11.7, 'dewpt': 10.4, 'rel_hum': 93, 'delta_t': 0.6, 'wind_dir': 'CALM', 'wind_spd_kmh': 0, 'wind_spd_kt': 0, 'gust_kmh': 0, 'gust_kt': 0, 'press': 1031.5, 'rain_trace': 0.2}}]}})

In [11]:
body={"doc": {"fix_pos": {
    "lat":-37.8,
    "lon":145
}}}
client.update(index="bom_weather-000001", id="IOQHNI8Bj0R9TQ6uLc3V",body=body)

ObjectApiResponse({'_index': 'bom_weather-000001', '_id': 'IOQHNI8Bj0R9TQ6uLc3V', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 4416, '_primary_term': 1})

In [16]:
body = {
    "query": {
        "bool":
            {
                "must":{
                    "match":{
                    "_id":"IOQHNI8Bj0R9TQ6uLc3V"
                }
                }
            }},
    "script": {
        "inline": "ctx._source.category = params.category",
        "params": {
            "fix_pos": {
    "lat":-37.8,
    "lon":145.6
}
        },
        "lang": "painless"
    }
}

client.update_by_query(index="bom_weather", body=body)

ObjectApiResponse({'took': 3764, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [24]:
allQuery ={
    "query":
        {
            "match_all":{}
        },
    "size":2000
}
result = client.search(
        index= "bom_weather-000001",
        body= allQuery,
        scroll="1m"
    )

In [19]:
result

ObjectApiResponse({'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoAxZKMGM1ZnFGUlJHS0dlbWVxYzNtaG9BAAAAAACAK28WbWJLMUtacHVReUNVR2pwOFp5X2p0QRZKMGM1ZnFGUlJHS0dlbWVxYzNtaG9BAAAAAACAK3AWbWJLMUtacHVReUNVR2pwOFp5X2p0QRZLeGxxWUE3OFN4T0ZpODUzeHQzLTZ3AAAAAACLSmQWd20wMWNCbmFTbjZQMVBSWEZfa3BVQQ==', 'took': 372, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 13155, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'bom_weather-000001', '_id': 'IeQHNI8Bj0R9TQ6uLc3v', '_score': 1.0, '_source': {'STATION': 'Melbourne (Olympic Park)', 'STATE': 'Victoria', 'DATE': '2024-04-30T23:00:00', 'Location': {'lat': -37.8, 'lon': 145.0}, 'wmo': 95936, 'air_temp': 11.1, 'apparent_t': 11.2, 'dewpt': 10.3, 'rel_hum': 95, 'delta_t': 0.4, 'wind_dir': 'CALM', 'wind_spd_kmh': 0, 'wind_spd_kt': 0, 'gust_kmh': 0, 'gust_kt': 0, 'press': 1031.6, 'rain_trace': 0.2}}, {'_index': 'bom_weather-000001', '_id': 'I-QHNI8Bj0R9TQ6uLs0p', '_

In [20]:
len(result["hits"]["hits"])

1000

In [28]:
result = client.search(
        index= "bom_weather",
        body= allQuery,
        scroll="1m"
    )
info = []
hits = result["hits"]["hits"]
cnt=0
for hit in hits:
    idx = hit["_index"]
    id = hit["_id"]
    lat = hit["_source"]['Location']['lat']
    lon = hit["_source"]['Location']['lon']
    info.append((idx,id,lat,lon))
    cnt+=1
print(cnt)
while len(hits) > 0:
    scroll_id = result["_scroll_id"]
    result = client.scroll(scroll_id=scroll_id, scroll="1m")
    hits = result["hits"]["hits"]
    for hit in hits:
        idx = hit["_index"]
        id = hit["_id"]
        lat = hit["_source"]['Location']['lat']
        lon = hit["_source"]['Location']['lon']
        info.append((idx,id,lat,lon))
        cnt+=1
    print(cnt)

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
39500
39500


In [29]:
len(info)

39500

In [ ]:
from tqdm import tqdm
for i in tqdm(info):

    body={"doc": {"fix_pos": {
    "lat":i[2],
    "lon":i[3]
    }}}
    client.update(index=i[0], id=i[1],body=body)

 49%|████▉     | 19462/39500 [2:16:45<18:42, 17.85it/s]   